In [2]:
import mysql.connector
import pandas as pd
import unicodedata
import numpy as np


---

<h3><center>Funciones</center></h3>

In [2]:
def conectar_mysql():
    try:
        conexion = mysql.connector.connect(
            host='localhost',
            user='root',
            password='1533542415',
            database='quantyle_analitics'
        )
        print("¡Conexión establecida correctamente!")
        return conexion
    except mysql.connector.Error as error:
        print("Error al conectarse a MySQL:", error)
        return None
    

---

In [3]:
# Establecer la conexión con la base de datos
conexion = conectar_mysql()

# Verificar si la conexión se realizó con éxito
if conexion.is_connected():
    print("Conexión exitosa")

# Realizar operaciones en la base de datos, por ejemplo, ejecutar una consulta SQL
cursor = conexion.cursor()

# Ejemplo de consulta SQL
consulta = "SELECT * FROM categories_yelp"

# Ejecutar la consulta
cursor.execute(consulta)

# Obtener los resultados
resultados = cursor.fetchall()

# Mostrar los resultados
for fila in resultados:
    print(fila)

# Cerrar el cursor y la conexión
cursor.close()
conexion.close()


¡Conexión establecida correctamente!
Conexión exitosa


# 1) State

In [4]:
state = pd.read_parquet(r'..\datasets\processed\bd\1_states.parquet.gz')

conexion = conectar_mysql()

# Insertar filas en la tabla state
cursor = conexion.cursor()
for index, row in state.iterrows():
    cursor.execute("INSERT INTO state (state) VALUES (%s)", (row['state'],))  # Asegúrate de tener la coma al final

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 2) Categories

In [5]:
categories = pd.read_parquet(r'..\datasets\processed\bd\2_categories.parquet.gz')

# Aplicar strip() a la columna 'name' para eliminar espacios en blanco adicionales
categories['name'] = categories['name'].str.strip()

# Convertir la columna 'name' a minúsculas para uniformizar los datos
categories['name'] = categories['name'].str.lower()

# Eliminar las filas duplicadas basadas en la columna 'name'
categories.drop_duplicates(subset=['name'], keep='first', inplace=True)

conexion = conectar_mysql()

# Insertar filas en la tabla categories
cursor = conexion.cursor()
for index, row in categories.iterrows():
    cursor.execute("INSERT INTO categories (categories_id , name) VALUES (%s, %s)", (row['categories_id'], row['name'],))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 3) User yelp

In [6]:
user_yelp = pd.read_parquet(r'..\datasets\processed\bd\3_user_yelp.parquet.gz')
user_yelp['creation'] = user_yelp['creation'].dt.strftime('%Y-%m-%d %H:%M:%S')

conexion = conectar_mysql()

# Insertar filas en la tabla bussiness_yelp
cursor = conexion.cursor()
for index, row in user_yelp.iterrows():
    cursor.execute("INSERT INTO user_yelp (user_id, name, creation, review_count, influence, stars) VALUES (%s, %s, %s, %s, %s, %s)",
                (row['user_id'], row['name'], row['creation'], row['review_count'], row['influence'], row['stars']))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 4) User google

In [7]:
user_google = pd.read_parquet(r'..\datasets\processed\bd\4_user_google.parquet.gz')
user_google['creation'] = user_google['creation'].dt.strftime('%Y-%m-%d %H:%M:%S')
user_google.drop_duplicates(inplace=True)
user_google['user_id'] = user_google['user_id'].astype(str)
user_google['name'] = user_google['name'].replace(r'[^a-zA-Z\s]', '', regex=True)
user_google.drop(2023700,inplace=True)
user_google.dropna(inplace=True)
user_google = user_google.groupby(['user_id','name','creation','review_count'])['stars'].mean().reset_index()
user_google = user_google.drop_duplicates(subset=['user_id'], keep='first')


def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in user_google.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    user_google[column] = user_google[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla bussiness_yelp
cursor = conexion.cursor()
for index, row in user_google.iterrows():
    cursor.execute("INSERT INTO user_google (user_id, name, creation, review_count, stars) VALUES (%s, %s, %s, %s, %s)",
                (row['user_id'], row['name'], row['creation'], row['review_count'], row['stars']))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()




¡Conexión establecida correctamente!


# 5) Bussiness google

In [8]:
categories_google = pd.read_parquet(r'..\datasets\processed\bd\7_categories_google.parquet.gz')
business_google = pd.read_parquet(r'..\datasets\processed\bd\5_bussiness_google.parquet.gz')


business_google.rename(columns={'state': 'state_id'}, inplace=True)
business_google["state_id"] = business_google["state_id"].replace("Florida", 1)
business_google["state_id"] = business_google["state_id"].replace("California", 2)
business_google["state_id"] = business_google["state_id"].replace("Illinois", 3)
business_google["state_id"] = business_google["state_id"].replace("New Jersey", 4)
business_google['name'] = business_google['name'].replace(r'[^a-zA-Z\s]', '', regex=True)
business_google.dropna(inplace=True)
business_google = business_google.drop_duplicates(subset=['gmap_id'], keep='first')
business_google.drop(22003,inplace=True)
business_google['avg_stars'] = business_google['avg_stars'].round(2)

business_google = business_google[business_google['gmap_id'].isin(categories_google['gmap_id'].unique())]


def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in business_google.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    business_google[column] = business_google[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in business_google[['gmap_id', 'name', 'latitude', 'longitude', 'avg_stars', 'state_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO business_google (gmap_id, name, latitude, longitude, avg_stars, state_id) VALUES (%s, %s, %s, %s, %s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 6) Business yelp

In [9]:
business_yelp = pd.read_parquet(r'..\datasets\processed\bd\6_bussiness_yelp.parquet.gz')

business_yelp.rename(columns={'state': 'state_id'}, inplace=True)
business_yelp.rename(columns={'stars': 'avg_stars'}, inplace=True)
business_yelp["state_id"] = business_yelp["state_id"].replace("Florida", 1)
business_yelp["state_id"] = business_yelp["state_id"].replace("California", 2)
business_yelp["state_id"] = business_yelp["state_id"].replace("Illinois", 3)
business_yelp["state_id"] = business_yelp["state_id"].replace("New Jersey", 4)
business_yelp['name'] = business_yelp['name'].replace(r'[^a-zA-Z\s]', '', regex=True)
business_yelp.dropna(inplace=True)
business_yelp = business_yelp.drop_duplicates(subset=['business_id'], keep='first')
business_yelp['avg_stars'] = business_yelp['avg_stars'].round(2)

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in business_yelp.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    business_yelp[column] = business_yelp[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in business_yelp[['business_id', 'name', 'latitude', 'longitude', 'avg_stars', 'state_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO business_yelp (business_id, name, latitude, longitude, avg_stars, state_id) VALUES (%s, %s, %s, %s, %s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 7) Categories google

In [10]:
categories_google = pd.read_parquet(r'..\datasets\processed\bd\7_categories_google.parquet.gz')

categories_google = categories_google[categories_google['categories_id'].isin(categories['categories_id'])]

categories_google = categories_google[categories_google['gmap_id'].isin(business_google['gmap_id'])]

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in categories_google.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    categories_google[column] = categories_google[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)
    
conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in categories_google[['gmap_id', 'categories_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO categories_google (gmap_id, categories_id) VALUES (%s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 8) Categories yelp

In [11]:
categories_yelp = pd.read_parquet(r'..\datasets\processed\bd\8_categories_yelp.parquet.gz')

categories_yelp = categories_yelp[categories_yelp['categories_id'].isin(categories['categories_id'])]

categories_yelp = categories_yelp[categories_yelp['business_id'].isin(business_yelp['business_id'])]

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in categories_yelp.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    categories_yelp[column] = categories_yelp[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in categories_yelp[['business_id', 'categories_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO categories_yelp (business_id, categories_id) VALUES (%s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 9) Reviews google

In [15]:
reviews_google = pd.read_parquet(r'..\datasets\processed\bd\9_reviews_google.parquet.gz')

reviews_google['date'] = reviews_google['date'].dt.strftime('%Y-%m-%d %H:%M:%S').str.replace('\r', '')
reviews_google['resp_date'] = reviews_google['resp_date'].dt.strftime('%Y-%m-%d %H:%M:%S').str.replace('\r', '')

reviews_google['user_id'] = reviews_google['user_id'].astype(str)

categories_yelp = categories_yelp[categories_yelp['categories_id'].isin(categories['categories_id'])]

reviews_google.loc[reviews_google['resp_date'].isna(), 'resp_date'] = reviews_google.loc[reviews_google['resp_date'].isna(), 'date']


def round_resp_sentiment(value):
    if value < -0.5:
        return 0
    elif -0.5 <= value < 0.5:
        return 1
    else:
        return 2


reviews_google['resp_sentiment'] = reviews_google['resp_sentiment'].apply(round_resp_sentiment)
reviews_google['resp_sentiment'] = reviews_google['resp_sentiment'].astype(str)


# export
# reviews_google.to_csv(r'reviews_google.csv',encoding='utf-8')



LOAD DATA INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\reviews_google.csv'
INTO TABLE reviews_google
CHARACTER SET utf8
FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n'
IGNORE 1 LINES
(review_id, user_id, gmap_id, stars, sentiment, date, resp_sentiment, resp_date);


# 10) Reviews yelp

In [32]:
reviews_yelp = pd.read_parquet(r'..\datasets\processed\bd\10_reviews_yelp.parquet.gz')

reviews_yelp['date'] = reviews_yelp['date'].dt.strftime('%Y-%m-%d %H:%M:%S').str.replace('\r', '')

reviews_yelp = reviews_yelp[reviews_yelp['business_id'].isin(business_yelp['business_id'])]

reviews_yelp = reviews_yelp[reviews_yelp['user_id'].isin(user_yelp['user_id'])]


reviews_yelp.to_csv(r'reviews_yelp.csv',encoding='utf-8', index=False)



In [33]:
reviews_yelp

,review_id,user_id,business_id,stars,sentiment,date
0,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,2,2016-07-25 07:31:06
1,qRhOkdYNO1URgn1WJfK1cg,59MxRhNVhU9MYndMkz0wtw,W7gSJz80DywKnPRIGjA2Bw,5.0,2,2016-07-25 07:20:23
2,ZZbpYMY4s8sVQGEU1jAuVA,59MxRhNVhU9MYndMkz0wtw,l_slvEnh4v3W8BXF1gYlcQ,5.0,2,2016-07-23 00:13:36
3,_Ub20uO1MKy4XOVPOdzpqw,lUYboGI6aFbZ0dX27pijpA,gebiRewfieSdtt17PTW6Zg,1.0,1,2017-06-28 01:04:59
4,-DjIfoNFAiT5J4kF9hXocQ,SrfDRvGKI8FQq9LCr0dQuQ,gebiRewfieSdtt17PTW6Zg,3.0,1,2017-01-14 23:31:35
...,...,...,...,...,...,...
909981,HVYhSbKGyj2R39fTDY7sGg,7KBaoiKUhdTvOsRFOMGwZw,VnAJnVpXHOIBdg6qFJBc3g,1.0,1,2021-08-19 02:30:16
909982,CzJMAaOokvASWgVZg8B__w,L-h5y32VWEV60QXuDPW-hg,VnAJnVpXHOIBdg6qFJBc3g,1.0,1,2021-09-12 05:20:37
909983,9_ztYeoSwdz7S9TW8xkDQA,03q-tEfa2aJtKhf00ZZ-hg,VnAJnVpXHOIBdg6qFJBc3g,1.0,1,2021-04-25 13:44:37
909984,jDeNxby0ZI5UMsZzrywedA,NIlmQ38hxTR2w6lNSzJEuQ,VnAJnVpXHOIBdg6qFJBc3g,1.0,1,2021-10-06 01:52:41


In [ ]:
'''LOAD DATA INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\reviews_google.csv'
INTO TABLE reviews_google
CHARACTER SET utf8
FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n'
IGNORE 1 LINES
(review_id, user_id, gmap_id, stars, sentiment, date, resp_sentiment, resp_date);





LOAD DATA INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\reviews_yelp.csv'
INTO TABLE reviews_yelp
CHARACTER SET utf8
FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n'
IGNORE 1 LINES
(review_id, user_id, business_id, stars, sentiment, date);'''


